In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from core.pandas_utils import series_start_end_diff
from  transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries, ProcessedTimeSeries
from transform.raw_results.config import *
from transform.processed_results.main import make_soh_presentable_per_vehicle
from core.caching_utils import cache_result

from transform.raw_results.mercedes_results import *

Notebook pour forcer la décroissance des SoH 

## Import data

In [ ]:
@cache_result("../tesla/tesla_results.parquet", on="local_storage")
def get_results():
    results = (
        TeslaProcessedTimeSeries("tesla", force_update=False)
        .query("trimmed_in_charge")
        .groupby(["vin", "trimmed_in_charge_idx"])
        .agg(
            #energy_added=pd.NamedAgg("charge_energy_added", series_start_end_diff),
            energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
            energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            charging_power=pd.NamedAgg("charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
        )
        .reset_index(drop=False)
        .eval("energy_added = energy_added_end - energy_added_min")
        .eval("soh = energy_added / (soc_diff / 100.0 * capacity)")
        .query("soc_diff > 40 & soh.between(0.75, 1.05)")
        .eval("level_1 = soc_diff * (charging_power < @LEVEL_1_MAX_POWER) / 100")
        .eval("level_2 = soc_diff * (charging_power.between(@LEVEL_1_MAX_POWER, @LEVEL_2_MAX_POWER)) / 100")
        .eval("level_3 = soc_diff * (charging_power > @LEVEL_2_MAX_POWER) / 100")
	.eval("bottom_soh = soh.between(0.75, 0.9)")
        .eval("fixed_soh_min_end = soh.mask(tesla_code == 'MTY13', soh / 0.96)")
        .eval("fixed_soh_min_end = fixed_soh_min_end.mask(bottom_soh & tesla_code == 'MTY13', fixed_soh_min_end + 0.08)")
        .eval("soh = fixed_soh_min_end")
        .sort_values(["tesla_code", "vin", "date"])
    )
    return results

df = get_results()

In [ ]:
# LRW3E7FA4LC103400 -> vin pour voir appliqué
# XP7YGCES7RB428560 -> vin pour lequel on n'effectue rien
df_vin = df[df['vin']=="LRW3E7FA4LC103400"].copy()


## Appliquer un decroissance avec un calcul de windows fonction 

Remplace ce qui est fait aujourd'hui avec la fonction `force_monotonic_decrease`

In [ ]:
def force_decay(df):
    rolling_mean = df["soh"].rolling(window=3, min_periods=1).mean()

    # Forcer la monotonie strictement décroissante 
    rolling_decreasing = rolling_mean.cummin()
    soh_decreasing = rolling_decreasing  - (df["odometer"] - df["odometer"].min()) * 1e-8
    return soh_decreasing

In [ ]:
df_vin['rolling_soh'] = force_decay(df_vin)

In [ ]:
df_vin[['odometer', 'soh', 'rolling_soh']]

## Plot 

In [ ]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=df_vin['odometer'], y=df_vin['soh'], mode="markers", opacity=1, name='get result'), )
fig.add_trace(go.Scatter(x=df_vin['odometer'], y=df_vin['rolling_soh'], mode="markers", opacity=1, name='rolling'), )
